# Nowcasting PDIR data

This notebook provide instructions on how to nowcast using the data that was previously downloaded. This notebook will cover how to 

1. Use the dataloader from the h5 files to produce windowed inputs
2. Initialize config files for each nowcasting model
3. Nowcast the generated inputs and save the outputs in a format ready to be used for downstream tasks

Author: Akshay Aravamudan (aaravamudan2014@my.fit.edu)

Last Edited: Jan 21, 2025

## Imports

In [1]:
from servir.utils.evaluation import generate_outputs
from servir.core.data_provider import IMERGDataModule
import pandas as pd


csv_filename = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/flood_events_PDIR.csv'

data_df = pd.read_csv(csv_filename)
data_df['simulation for nowcast'] = pd.to_datetime(data_df['simulation for nowcast'])
data_df['simulation for nowcast'].values

Pysteps configuration file found at: /volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/pysteps/pystepsrc



/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array(['2020-06-14T08:00:00.000000000', '2020-06-18T02:00:00.000000000',
       '2020-06-04T15:00:00.000000000', '2018-07-24T07:00:00.000000000',
       '2020-10-10T09:00:00.000000000', '2020-10-10T08:00:00.000000000',
       '2020-09-05T00:00:00.000000000', '2019-08-02T06:00:00.000000000',
       '2020-06-14T05:00:00.000000000', '2018-07-15T20:00:00.000000000',
       '2018-07-24T07:00:00.000000000', '2020-06-18T08:00:00.000000000',
       '2019-08-02T10:00:00.000000000'], dtype='datetime64[ns]')

## Download data for all events


In [2]:
from servir.utils.PERSIAN_downloader import download_files
import numpy as np
from servir.utils.download_data import initialize_event_folders
    
    
# download_files(input_date_str, save_dir , input_tiff_dir, output_tiff_dir_name)
for index, start_date in enumerate(data_df['simulation for nowcast'].values):
    # download_files
    # print(start_date.strftime("YYYY-mm-dd"))
    try:
        date_str = np.datetime_as_string(start_date, unit='h')
        date_str = date_str.replace('T', ':')
        save_dir = "/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/"
        raw_data_name = "PDIR_data_raw_"+str(index)
        input_tiff_name = "PDIR_data_tiff_"+str(index)
        output_tiff_dir_name = "PDIR_data_processed_"+str(index)
        final_save_input_folder = initialize_event_folders(folder_name = save_dir,event_id = raw_data_name )
        final_save_input_tiff_folder = initialize_event_folders(folder_name = save_dir,event_id = input_tiff_name )
        
        final_save_output_folder = initialize_event_folders(folder_name = save_dir,event_id = output_tiff_dir_name )

        # save_dir , input_tiff_dir, output_tiff_dir_name

        download_files(input_date_str=date_str, save_dir = final_save_input_folder , input_tiff_dir = final_save_input_tiff_folder , output_tiff_dir_name=final_save_output_folder, event_id="PDIR_data_"+str(index))
    
    except:
        print("Failed to download for event {}, continuing".format(str(index)))
        

Download complete: pdirnow1h20061400.bin.gz
Decompressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_0/pdirnow1h20061400.bin
Removed compressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_0/pdirnow1h20061400.bin.gz
Download complete: pdirnow1h20061401.bin.gz
Decompressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_0/pdirnow1h20061401.bin
Removed compressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_0/pdirnow1h20061401.bin.gz
Download complete: pdirnow1h20061402.bin.gz
Decompressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_0/pdirnow1h20061402.bin
Removed compressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_0/pdirnow1h20061402.bin.gz
Download complete: pdirnow1h20061403.bin.gz
Decompressed file: /vol_efthy

/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


------h5 FILE PRODUCED----------
All processing complete. Folders have been removed.
Download complete: pdirnow1h20061718.bin.gz
Decompressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_1/pdirnow1h20061718.bin
Removed compressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_1/pdirnow1h20061718.bin.gz
Download complete: pdirnow1h20061719.bin.gz
Decompressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_1/pdirnow1h20061719.bin
Removed compressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_1/pdirnow1h20061719.bin.gz
Download complete: pdirnow1h20061720.bin.gz
Decompressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_1/pdirnow1h20061720.bin
Removed compressed file: /vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_raw_1/pdirnow1h2006

## Run nowcasts for models

In [ ]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker



method_name = 'steps' # steps, lagrangian, linda
for index, start_date in enumerate(data_df['simulation for nowcast'].values):
    try:
        # event id for which the data was downloaded
        event_id = 'PDIR_data_'+str(index)

        # location of the h5 file that was generated after downloading the data
        h5_dataset_location = '../data/events/'+str(event_id)+'.h5'

        # as of now, we do not have IR data, so we set it None
        ir_h5_dataset_location = None

        # this string is used to determine the kind of dataloader we need to use
        # for processing individual events, we reccommend the user to keep this fixed
        dataset_type = 'wa'
        
        data_provider =  IMERGDataModule(
            forecast_steps = 12,
            history_steps = 8,
            imerg_filename = h5_dataset_location,
            ir_filename = ir_h5_dataset_location,
            batch_size = 32,
            image_shape = (360, 516),
            normalize_data=False,
            dataset = dataset_type,
            production_mode = True)
        print(data_provider)
        # for now we are treating the test dataloader as the main one since we are only interested in predicting for individual events
        data_loader = data_provider.test_dataloader()
        
        if method_name == 'steps':
            model_type = 'steps'
            model_config_location = 'configs/wa_imerg/PySTEPS.py'
            model_save_location = ''
            use_gpu = False
            produce_ensemble_outputs = False
            model_output_location = 'results/'
            output = generate_outputs(data_loader, model_type, model_config_location, 
                                            model_save_location, use_gpu, 
                                            produce_ensemble_outputs=produce_ensemble_outputs, 
                                            event_name = event_id, model_output_location = model_output_location)
        elif method_name == 'lagrangian':
            model_type = 'lagrangian'
            model_config_location = 'configs/wa_imerg/lagrangian_persistence.py'
            model_save_location = ''
            use_gpu = False
            produce_ensemble_outputs = False
            model_output_location = 'results/'
            output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)
        elif method_name == 'linda':
            model_type = 'linda'
            model_config_location = 'configs/wa_imerg/LINDA.py'
            model_save_location = ''
            use_gpu = False
            produce_ensemble_outputs = False
            model_output_location = 'results/'
            output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)

        # tif file per prediction

        model_picker = ModelPicker(model_type=model_type, 
                                    model_config_location=model_config_location, 
                                    model_save_location=model_save_location,
                                    use_gpu=use_gpu)
        model_picker.load_data(h5_dataset_location)

        model_picker.load_model(get_ensemble=False)

        # predictions = model_picker.predict()
        
        print(output.shape)
        model_picker.save_output(str(event_id)+'_outputs.h5', output, num_predictions=len(output), dataset= 'PDIR')

        h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
         meta_fname = 'PDIR_metadata.json', 
         tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/PDIR_data_processed_{}/predictions/'.format(str(index)),
         num_predictions=len(output),
         method=method_name,
         dataset = 'PDIR')

    except:
        pass


original shape (8, 900, 1295)
Precipitation Dataset input shape:  (1, 8, 1, 900, 1295)
loading config from configs/wa_imerg/LINDA.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
Computing LINDA nowcast
-----------------------

Inputs
------
dimensions:           900x1295
number of time steps: 8

Methods
-------
nowcast type:         deterministic
feature detector:     blob
extrapolator:         semilagrangian
kernel type:          anisotropic

Parameters
----------
number of time steps:       12
ARI model order:            1
localization window radius: 180.0

Detecting features... found 15 blobs.
Transforming to Lagrangian coordinates... done.
Estimating the first convolution kernel... done.
Estimating the ARI(p,1) parameters... done.
Estimating the second convolution kernel... done.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
Computing nowcast for time step 8... done.
Comput

/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
Computing LINDA nowcast
-----------------------

Inputs
------
dimensions:           900x1295
number of time steps: 8

Methods
-------
nowcast type:         deterministic
feature detector:     blob
extrapolator:         semilagrangian
kernel type:          anisotropic

Parameters
----------
number of time steps:       12
ARI model order:            1
localization window radius: 180.0

Detecting features... found 15 blobs.
Transforming to Lagrangian coordinates... done.
Estimating the first convolution kernel... done.
Estimating the ARI(p,1) parameters... done.
Estimating the second convolution kernel... done.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
Computing nowcast for time step 8... done.
Comput

...

starting predictions for batch 0
Computing LINDA nowcast
-----------------------

Inputs
------
dimensions:           900x1295
number of time steps: 8

Methods
-------
nowcast type:         deterministic
feature detector:     blob
extrapolator:         semilagrangian
kernel type:          anisotropic

Parameters
----------
number of time steps:       12
ARI model order:            1
localization window radius: 180.0

Detecting features... found 15 blobs.
Transforming to Lagrangian coordinates... done.
Estimating the first convolution kernel... done.
Estimating the ARI(p,1) parameters... done.
Estimating the second convolution kernel... done.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
Computing nowcast for time step 8... done.
Comput

/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
Computing LINDA nowcast
-----------------------

Inputs
------
dimensions:           900x1295
number of time steps: 8

Methods
-------
nowcast type:         deterministic
feature detector:     blob
extrapolator:         semilagrangian
kernel type:          anisotropic

Parameters
----------
number of time steps:       12
ARI model order:            1
localization window radius: 180.0

Detecting features... found 15 blobs.
Transforming to Lagrangian coordinates... done.
Estimating the first convolution kernel... done.
Estimating the ARI(p,1) parameters... done.
Estimating the second convolution kernel... done.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
Computing nowcast for time step 8... done.
Comput

/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
Computing LINDA nowcast
-----------------------

Inputs
------
dimensions:           900x1295
number of time steps: 8

Methods
-------
nowcast type:         deterministic
feature detector:     blob
extrapolator:         semilagrangian
kernel type:          anisotropic

Parameters
----------
number of time steps:       12
ARI model order:            1
localization window radius: 180.0

Detecting features... found 15 blobs.
Transforming to Lagrangian coordinates... done.
Estimating the first convolution kernel... done.
Estimating the ARI(p,1) parameters... done.
Estimating the second convolution kernel... done.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
Computing nowcast for time step 8... done.
Comput

## Initialize parameters

In [2]:
# event id for which the data was downloaded
event_id = 'PDIR_data'

# location of the h5 file that was generated after downloading the data
h5_dataset_location = '../data/events/'+str(event_id)+'.h5'

# as of now, we do not have IR data, so we set it None
ir_h5_dataset_location = None

# this string is used to determine the kind of dataloader we need to use
# for processing individual events, we reccommend the user to keep this fixed
dataset_type = 'wa'

## Initialize the dataloader

In [3]:
data_provider =  IMERGDataModule(
        forecast_steps = 12,
        history_steps = 8,
        imerg_filename = h5_dataset_location,
        ir_filename = ir_h5_dataset_location,
        batch_size = 32,
        image_shape = (360, 516),
        normalize_data=False,
        dataset = dataset_type,
        production_mode = True)

# for now we are treating the test dataloader as the main one since we are only interested in predicting for individual events
data_loader = data_provider.test_dataloader()

original shape (9, 360, 516)
Precipitation Dataset input shape:  (0, 8, 1, 360, 516)


## Initialize config files for individual models and predict

For producing teh nowcasts using each of the models, the following parameters need to be initialized

1. model_type: name of the model ('convlstm', 'linda', 'steps', 'lagrangian', 'naive')
2. model_config_location: location where the python file with model specific parameters can be found. These have already been populated in the `configs/wa_imerg/` directory.
3. model_save_location: if the model requires a saved object (for instance neural network weights), this parameter points to that location.
4. use_gpu: flag for whether the model should use a GPU. note that this is only applicable for neural network models and the node which runs this notebook must contain a GPU.


### ConvLSTM

In [ ]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'convlstm'
model_config_location = 'configs/wa_imerg/ConvLSTM.py'
model_save_location = 'temp/imerg_only_mse_params.pth'
model_output_location = 'results/'
use_gpu = False
produce_ensemble_outputs = False
convlstm_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id, model_output_location=model_output_location)

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

# predictions = model_picker.predict()

model_picker.save_output(str(event_id)+'_outputs.h5', convlstm_output, num_predictions=len(convlstm_output))

# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(convlstm_output))


loading config from configs/wa_imerg/ConvLSTM.py ...
loading config from configs/wa_imerg/ConvLSTM.py ...


### STEPS

In [6]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'steps'
model_config_location = 'configs/wa_imerg/PySTEPS.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
steps_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)
# tif file per prediction

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

# predictions = model_picker.predict()

model_picker.save_output(str(event_id)+'_outputs.h5', steps_output, num_predictions=len(steps_output))
# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(steps_output),
#          method='steps')

loading config from configs/wa_imerg/PySTEPS.py ...


loading config from configs/wa_imerg/PySTEPS.py ...


### Lagrangian Persistence

In [8]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'lagrangian'
model_config_location = 'configs/wa_imerg/lagrangian_persistence.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
lagrangian_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

lagrangian_output = model_picker.predict()

# lagrangian_output = lagrangian_output[None, :, :, :]
model_picker.save_output(str(event_id)+'_outputs.h5', lagrangian_output, num_predictions=len(lagrangian_output))

print(lagrangian_output.shape)
# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(lagrangian_output))



loading config from configs/wa_imerg/lagrangian_persistence.py ...


loading config from configs/wa_imerg/lagrangian_persistence.py ...
(12, 900, 1295)


### Naive Persistence

In [9]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'naive'
model_config_location = 'configs/wa_imerg/naive_persistence.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
naive_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

model_picker.save_output(str(event_id)+'_outputs.h5', naive_output, num_predictions=len(naive_output))

# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(naive_output))

loading config from configs/wa_imerg/naive_persistence.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.008693218231201172  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002703428268432617  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0031270980834960938  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002671480178833008  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0027790069580078125  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0027272701263427734  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.003042459487915039  seconds
loading config from configs/wa_imerg/naive_persistence.py ...


### LINDA

In [ ]:
model_type = 'linda'
model_config_location = 'configs/wa_imerg/LINDA.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
naive_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)
